<a href="https://colab.research.google.com/github/leondb999/4-sem-picolo/blob/master/Titantic_Neuronales_Netz_Neue_Konzepte_Sondermann.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Lösung von Leon Dickob

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import optimizers


In [ ]:
from google.colab import files
uploaded1 = files.upload()
uploaded2 = files.upload()
uploaded3 = files.upload()

import pandas as pd
import io

train_set = pd.read_csv(io.StringIO(uploaded1['train.csv'].decode('utf-8')))
test_set = pd.read_csv(io.StringIO(uploaded2['test.csv'].decode('utf-8')))
output_set = pd.read_csv(io.StringIO(uploaded3['gender_submission.csv'].decode('utf-8')))
train_set

Saving train.csv to train.csv


Saving test.csv to test.csv


Saving gender_submission.csv to gender_submission.csv


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
train_set = train_set[ train_set['Embarked'] != 0 ]

In [ ]:

x_train = train_set.drop(['PassengerId','Name','Ticket','Survived', 'Cabin'], axis=1)
y_train = train_set['Survived']
x_test = test_set.drop(['PassengerId','Name','Ticket', 'Cabin'], axis=1)

x_train['Age'] = x_train['Age'].fillna(x_train['Age'].mean())
x_test['Age'] = x_test['Age'].fillna(x_test['Age'].mean())

x_train = x_train.fillna(float(0.0))
x_test = x_test.fillna(float(0.0))
x_test.head()


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,34.5,0,0,7.8292,Q
1,3,female,47.0,1,0,7.0000,S
2,2,male,62.0,0,0,9.6875,Q
3,3,male,27.0,0,0,8.6625,S
4,3,female,22.0,1,1,12.2875,S


In [ ]:
x_train = pd.get_dummies(x_train, columns= ['Sex','Embarked']) 
x_test = pd.get_dummies(x_test, columns= ['Sex','Embarked'])
x_train = x_train.drop(['Embarked_0.0'], axis=1)
print(x_test.shape)
x_train.shape


(418, 10)


(891, 10)

In [ ]:
from keras import models
from keras import layers
from tensorflow.keras import optimizers
from keras import losses 
from keras import metrics


model = models.Sequential()

model.add(layers.Dense(32, activation='relu', input_shape=(10,)))
model.add(layers.Dense(16, activation='relu', input_shape=(10,)))
model.add(layers.Dense(8, activation='relu', input_shape=(10,)))
model.add(layers.Dense(1, activation='sigmoid', input_shape=(10,)))

model.compile(optimizer=optimizers.RMSprop(lr=0.001),loss=losses.binary_crossentropy,metrics=[metrics.binary_accuracy])

model.fit(x_train, y_train, epochs=80)


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/optimizer_v2.py:356: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  "The `lr` argument is deprecated, use `learning_rate` instead.")


Epoch 1/80
28/28 [==============================] - 1s 3ms/step - loss: 4.2597 - binary_accuracy: 0.5870
Epoch 2/80
28/28 [==============================] - 0s 2ms/step - loss: 0.6777 - binary_accuracy: 0.6756
Epoch 3/80
28/28 [==============================] - 0s 2ms/step - loss: 0.6290 - binary_accuracy: 0.6813
Epoch 4/80
28/28 [==============================] - 0s 2ms/step - loss: 0.6031 - binary_accuracy: 0.6902
Epoch 5/80
28/28 [==============================] - 0s 2ms/step - loss: 0.5870 - binary_accuracy: 0.7048
Epoch 6/80
28/28 [==============================] - 0s 2ms/step - loss: 0.5838 - binary_accuracy: 0.7059
Epoch 7/80
28/28 [==============================] - 0s 2ms/step - loss: 0.5715 - binary_accuracy: 0.7093
Epoch 8/80
28/28 [==============================] - 0s 2ms/step - loss: 0.5513 - binary_accuracy: 0.7329
Epoch 9/80
28/28 [==============================] - 0s 2ms/step - loss: 0.5537 - binary_accuracy: 0.7374
Epoch 10/80
28/28 [==============================] - 0s

In [ ]:
prediction = model.predict(x_test)
ids = test_set['PassengerId'].copy()
new_output = ids.to_frame()
new_output['Prediction']=prediction
s=output_set['Survived'].copy()
new_output['Real']=s


In [ ]:
df1 = pd.DataFrame(new_output,columns= ['Prediction', 'Real'])
df1['Übereinstimmung?'] = np.where(df1['Prediction'] == df1['Real'], 'True', 'False')                  

print(df1)

print(f"Anzahl Richtig: {len(df1[df1['Übereinstimmung?'].str.contains('True')])} || Anzahl Einträge: {len(prediction)} || Accuracy: {len(df1[df1['Übereinstimmung?'].str.contains('True')])/len(prediction)}") 
 

     Prediction  Real Übereinstimmung?
0      0.092611     0            False
1      0.370982     1            False
2      0.060180     0            False
3      0.085913     0            False
4      0.511466     1            False
..          ...   ...              ...
413    0.080852     0            False
414    0.968107     1            False
415    0.063141     0            False
416    0.080852     0            False
417    0.077848     0            False

[418 rows x 3 columns]
Anzahl Richtig: 0 || Anzahl Einträge: 418 || Accuracy: 0.0
